In [1]:
import bs4
import re
import requests
import pandas as pd
import sys

# results
df = pd.DataFrame(columns = ['link', 'title','author', 'cc'])

# arquivo baixado de: http://books.uc.pt/catalogo?all
text_file = open("../data/2023/celia/catalogo_all.html", "r")
data = text_file.read()
text_file.close()

In [10]:
def coleta(url):
    """ Função que acessa a URL de um livro e coleta os metadados do mesmo retorando um dicionário de dados"""
    res = requests.get(url)
    book = bs4.BeautifulSoup(res.text)

    # Título 
    title = book.find('h1').text

    # bloquinho direita
    bloquinho = book.find("p", {"class": ["small"] }).text
    linhas = bloquinho.splitlines()

    edicao = ''
    isbn = ''
    eisbn = ''
    doi = ''
    serie = ''
    data = ''

    for linha in linhas:
        if 'Edição' in linha:
            edicao = linha
        if 'ISBN' in linha:
            isbn = linha
        if 'eISBN' in linha:
            eisbn = linha
        if 'Série' in linha:
            serie = linha
        if 'DOI' in linha:
            doi = linha
        if 'Data:' in linha:
            data = linha
           
    # Palavras Chave
    keywords = book.find_all("p", {"class": ["small"] })[1].text
    keywords = keywords.replace("Palavras-Chaves", "")

    # Autores
    #authors = book.find('a', {'href': re.compile('author\?author')}).text 
    authors = book.find_all('a', {'href': re.compile('author\?author')})
    authors_string = ''
    afiliacao_string = ''
    for author in authors:
        authors_string = authors_string + ';' + author.text
        url_author = author.get('href')
        
        # Visitando a paǵina do autor para pegar a afiliação
        res_author = requests.get('http://books.uc.pt/' + url_author)
        author_page = bs4.BeautifulSoup(res_author.text)
        afiliacao = author_page.find('span', {"class": ["text-md","font-medium"] }).text
        afiliacao_string = afiliacao_string + ';' + afiliacao        

    # 0 não é CC e 1 é CC
    has_cc = book.find('svg', {'id': 'CC'})
    cc = 0 

    doi =  doi.replace("DOI: ", "")
    url_doi = requests.head('https://doi.org/'+doi, allow_redirects=True)
    url_redirected = url_doi.url
    google_play = 0
    amazon = 0
    if 'monographs' in url_redirected:
        monographs = requests.get(url_redirected)
        if "play.google.com/store" in monographs.text:
            google_play = 1
        if "www.amazon.com" in monographs.text:
            amazon = 1
   
    return {
        'url'   : url,
        'tilulo':    title,
        'autores':   authors_string,
        'afiliacao': afiliacao_string,
        'edicao': edicao,
        'isbn': isbn,
        'eisbn,': eisbn,
        'doi': doi,
        'serie': serie, 
        'data': data,
        'cc': cc,
        'keywords': keywords,
        'amazon': amazon,
        'google_play': google_play
    }

#url = 'http://books.uc.pt/book?book=1108'

#r = coleta(url)
#for keys,values in r.items():
#    print(keys, ":", values)

url : http://books.uc.pt/book?book=1108
tilulo : Plínio Salgado (1895-1975). Fascismo e autoritarismo no Brasil do Século XX
autores : ;João Fábio Bertonha
afiliacao : ;Universidade Estadual de Maringá
edicao : 1.ª Edição
isbn : eISBN: 978-989-26-2036-7
eisbn, : eISBN: 978-989-26-2036-7
doi : 
serie : Série: Outros Títulos
data : Data: Novembro, 2020
cc : 0
keywords : Plínio Salgado Ação Integralista Brasileira Fascismo e autoritarismo no Brasil Fascismo Autoritarismo   
	  
amazon : 0
google_play : 0


In [11]:
# Pegando os links de cada livro do catálogo geral e capturando os metadados de um por um

soup = bs4.BeautifulSoup(data)
links = soup.find_all("a", string = "LER +")

df = pd.DataFrame(columns = ['url','tilulo', 'autores','afiliacao', 'edicao', 'isbn','eisbn','doi','serie','data','cc','keywords','amazon','google_play'])
for link in links:

    href = link.get('href')
    url = 'http://books.uc.pt/' + href
    print(url)

    dict = coleta(url)
    
    # Adiciona no dataframe
    row = pd.DataFrame([dict])
    df = pd.concat([df, row])
    
df.to_csv('/tmp/celia.csv', index=False)

http://books.uc.pt/book?book=1324
http://books.uc.pt/book?book=1323
http://books.uc.pt/book?book=1325
http://books.uc.pt/book?book=1326
http://books.uc.pt/book?book=1322
http://books.uc.pt/book?book=1321
http://books.uc.pt/book?book=1318
http://books.uc.pt/book?book=1320
http://books.uc.pt/book?book=1316
http://books.uc.pt/book?book=1319
http://books.uc.pt/book?book=1308
http://books.uc.pt/book?book=1309
http://books.uc.pt/book?book=1310
http://books.uc.pt/book?book=1312
http://books.uc.pt/book?book=1313
http://books.uc.pt/book?book=1315
http://books.uc.pt/book?book=1314
http://books.uc.pt/book?book=1300
http://books.uc.pt/book?book=1305
http://books.uc.pt/book?book=1298
http://books.uc.pt/book?book=1299
http://books.uc.pt/book?book=1303
http://books.uc.pt/book?book=1297
http://books.uc.pt/book?book=1307
http://books.uc.pt/book?book=1288
http://books.uc.pt/book?book=1293
http://books.uc.pt/book?book=1291
http://books.uc.pt/book?book=1294
http://books.uc.pt/book?book=1289
http://books.u